# jsoncut ETL example for pandas

*json_normalize* in pandas is powerful, but it can generate a lot of unwanted columns of data for deeply nested JSON objects. A dataframe with hundreds of columns isn't much easier to examine than the original JSON object; especially if you're looking to whittle down a monster JSON to a handful of desired features.

*jsoncut* offers a more surgical approach, generating a list of numeric keys and their string key counterparts that show the heirarchical structure of your data at a glance and give you tools to transform only the keys you want into a new JSON object. As you'll see in the following examples *jsoncut* is both a command line tool and a package that you can bring into your project.

## A quick demo with fake MailChimp™ data

Let's pull in the modules we need to generate some data to work with in memory and to save out as a file.

In [71]:
import fake_mc 
# generate fake MailChimp contacts JSON 

from jsoncut.core import cut 
# pull in the core module from jsoncut

import pandas as pd

from pandas.io.json import json_normalize 
# for comparison and eventually creating our clean dataframe

import json 
# for saving out our dict of fake data as JSON

import subprocess 
# we'll use this to grab some output from jsoncut on the cmd line

Create a dict of 100 contacts with a root key of 'mc_contacts' and save it out as a .json file for later use.

In [2]:
contacts = fake_mc.fake_mc_list('mc_contacts', 100)

In [19]:
with open('contacts.json', 'w') as output:
    json.dump(contacts, output)

In [3]:
contacts['mc_contacts'][0]

{'_links': [{'href': 'https://us13.api.mailchimp.com/3.0/lists/25a058f03c/members/b8afd23e94db43cea51482df59cbe03c',
   'method': 'GET',
   'rel': 'self',
   'targetSchema': 'https://us13.api.mailchimp.com/schema/3.0/Definitions/Lists/Members/Response.json'},
  {'href': 'https://us13.api.mailchimp.com/3.0/lists/25a058f03c/members',
   'method': 'GET',
   'rel': 'parent',
   'schema': 'https://us13.api.mailchimp.com/schema/3.0/CollectionLinks/Lists/Members.json',
   'targetSchema': 'https://us13.api.mailchimp.com/schema/3.0/Definitions/Lists/Members/CollectionResponse.json'},
  {'href': 'https://us13.api.mailchimp.com/3.0/lists/25a058f03c/members/b8afd23e94db43cea51482df59cbe03c',
   'method': 'PATCH',
   'rel': 'update',
   'schema': 'https://us13.api.mailchimp.com/schema/3.0/Definitions/Lists/Members/PATCH.json',
   'targetSchema': 'https://us13.api.mailchimp.com/schema/3.0/Definitions/Lists/Members/Response.json'},
  {'href': 'https://us13.api.mailchimp.com/3.0/lists/25a058f03c/membe



So, our fist pass at this will be with *json_normalize*. 



In [4]:
contacts_df = json_normalize(contacts['mc_contacts'])

In [6]:
contacts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 26 columns):
_links                   100 non-null object
email_address            100 non-null object
email_client             100 non-null object
email_type               100 non-null object
id                       100 non-null object
ip_opt                   100 non-null object
ip_signup                100 non-null object
language                 100 non-null object
last_changed             100 non-null object
list_id                  100 non-null object
location.country_code    100 non-null object
location.dstoff          100 non-null int64
location.gmtoff          100 non-null int64
location.latitude        100 non-null float64
location.longitude       100 non-null float64
location.timezone        100 non-null object
member_rating            100 non-null int64
merge_fields.FNAME       100 non-null object
merge_fields.LNAME       100 non-null object
stats.avg_click_rate     100 non-null floa

In [7]:
contacts_df.shape

(100, 26)

The resulting DF has a shape of 100 x 26, so we've got a lot of columns. Powerful, but unless we want everything, we still have some cleaning up to do. 26 isn't so bad, but if you've got hundreds of columns and you only want a handful, there's a better way...

## *jsoncut* on the cmd line

So, let's quickly execute *jsoncut* as a shell command from within our notebook using the '!' prefix and view the options.

In [27]:
!jsoncut --help

Usage: jsoncut [OPTIONS] [JSONFILE]

  Quickly select or filter out properties in a JSON document.

Options:
  -r, --root TEXT                 Set the root of the JSON document
  -g, --get TEXT                  Get JSON key-values and/or elements
  -G, --getdefault <TEXT TEXT>...
                                  (key, default-value); same as get, except
                                  uses a default valuewhen the key or index is
                                  not found
  -d, --del TEXT                  delete JSON keys and/or indexes
  -l, --list                      numbered JSON keys list
  -i, --inspect                   inspect JSON document; all keys, indexes &
                                  types
  -c, --count                     count elements in top-level JSON arrays
  -f, --fullscan                  deep inpections
  -p, --fullpath                  preserve full path for names
  -q, --quotechar TEXT            set quoting char for keys
  -I, --indent INTEGER          

Now, let's set our root key as `mc_contacts` and `--list` the keys in `contacts.json`

In [84]:
!jsoncut --root mc_contacts contacts.json --list

 1 _links
 2 email_address
 3 email_client
 4 email_type
 5 id
 6 ip_opt
 7 ip_signup
 8 language
 9 last_changed
10 list_id
11 location
12 location.country_code
13 location.dstoff
14 location.gmtoff
15 location.latitude
16 location.longitude
17 location.timezone
18 member_rating
19 merge_fields
20 merge_fields.FNAME
21 merge_fields.LNAME
22 stats
23 stats.avg_click_rate
24 stats.avg_open_rate
25 status
26 timestamp_opt
27 timestamp_signup
28 unique_email_id
29 vip


And if you want to pull in the list of keys for later use, grab the output and make it into a DF.

In [85]:
keys_df = pd.DataFrame([x.strip().split() for x in 
 subprocess.getoutput('jsoncut --root mc_contacts contacts.json --list --nocolor').split('\n')],
                      index=None, columns=['numeric_key','string_key'])

In [86]:
keys_df

,numeric_key,string_key
0,1,_links
1,2,email_address
2,3,email_client
3,4,email_type
4,5,id
5,6,ip_opt
6,7,ip_signup
7,8,language
8,9,last_changed
9,10,list_id


Let's use `cut()` to grab just the keys we want, and while we're at it, in the *order* that we want them. In the following step, when we use *json_normalize*, we're now generating a clean DF with just the columns we need.

In [57]:
new_contacts = cut(contacts, rootkey='mc_contacts', getkeys='21,20,2,9,18,23,24,28')

In [58]:
new_contacts_df = json_normalize(new_contacts)

In [59]:
new_contacts_df.shape

(100, 8)

A little bit of housekeeping; lower-case the column names and cast the `last_changed` column to datetime.

In [87]:
new_contacts_df.columns = [x.lower() for x in new_contacts_df.columns]

In [67]:
new_contacts_df.last_changed = new_contacts_df.last_changed.astype('datetime64[ns]')

In [88]:
new_contacts_df.head(10)

,fname,lname,avg_click_rate,avg_open_rate,email_address,last_changed,member_rating,unique_email_id
0,Daniel,Little,0.5245,0.9403,kristinnguyen@yahoo.com,2018-02-28 00:37:10,4,72d685269e
1,Teresa,Chavez,0.1754,0.2404,chad77@yahoo.com,2018-03-06 03:01:03,1,1f8339dd6e
2,Troy,Thomas,0.1293,0.9713,jamesedward@hotmail.com,2018-03-13 22:59:36,5,c669f285e5
3,Corey,Mccormick,0.6081,0.4599,garciaglen@gmail.com,2018-01-11 08:47:57,3,92dc14af85
4,Teresa,Wallace,0.7382,0.3605,mosleyjacob@mcintyre.com,2018-03-20 18:12:30,3,5189fc6168
5,Mary,Baker,0.8410,0.6623,gcohen@brown.info,2018-03-09 22:13:42,2,d446a61091
6,Jennifer,Fox,0.6865,0.9967,qsimmons@bender-johnson.com,2018-03-06 03:10:22,4,b2e042a35c
7,Jamie,Soto,0.2643,0.8545,marissa37@gmail.com,2018-03-20 06:11:12,1,89e2ce549c
8,Rebecca,Wilson,0.9813,0.9946,mark77@yahoo.com,2018-01-30 13:08:39,3,c5796a5a3d
9,Michelle,Moore,0.2254,0.1506,nschmidt@west-davis.info,2018-04-08 18:06:10,0,52d60657ff


In [69]:
new_contacts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
FNAME              100 non-null object
LNAME              100 non-null object
avg_click_rate     100 non-null float64
avg_open_rate      100 non-null float64
email_address      100 non-null object
last_changed       100 non-null datetime64[ns]
member_rating      100 non-null int64
unique_email_id    100 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 6.3+ KB


And there you have it. *jsoncut* is a great tool for precision feature extraction from JSON objects and a great preparation step before bringing your data into *pandas*.